## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### 自定义一个agent

- 定义一个class
- 工具：默认搜索
- 提示词：定义agent要做什么任务
- outparse：约束LLM的行为和输出
- 不同的LLM不同的质量
### Customize an agent

- Define a class
- Tool: default search
- cue word: define what task the agent is to do
- outparse: constrains the behavior and output of the LLM
- Different qualities for different LLMs
<hr>

In [2]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain_openai import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
import os

- 设置agent可以使用的工具(集)
- Tools that can be used to set up agent (set)

In [3]:
class MyAgentTool:
    def __init__(self) -> None:
        self.serpapi = SerpAPIWrapper()
        
    def tools(self):
        return [
            Tool(
                name="search",
                description="适用于当你需要回答关于当前事件的问题时",
                func=self.serpapi.run,
            )
        ]
s = MyAgentTool()
s.serpapi.run("python")
    

[{'title': 'Huge python grabs Thai woman in her kitchen, squeezes her two hours before she can be freed',
  'link': 'https://apnews.com/article/thailand-snake-python-kitchen-0e2ad8b8fcdeeb9d8d17ffeb137fa85f',
  'source': 'AP News',
  'date': '18 hours ago',
  'thumbnail': 'https://serpapi.com/searches/66ed1008b9ab866f096a595b/images/05f4614e98818d4965b3f22aa7735b8d04382b2ebdfe89a5.jpeg'},
 {'title': 'Thai woman freed after hours of being strangled by a python',
  'link': 'https://www.cnn.com/2024/09/19/asia/thailand-woman-python-strangled-intl-hnk/index.html',
  'source': 'CNN',
  'date': '14 hours ago'},
 {'title': 'Woman, 64, Describes Fierce 2-Hour Struggle With 13-Foot Python',
  'link': 'https://www.yahoo.com/news/woman-64-describes-fierce-2-153016727.html',
  'source': 'Yahoo',
  'date': '1 day ago',
  'thumbnail': 'https://serpapi.com/searches/66ed1008b9ab866f096a595b/images/05f4614e98818d4902c774cbf0eb8555289293a5461a76a6.jpeg'},
 {'title': 'Florida’s 2024 Python Challenge winn

- 构建一个agents类
- Constructing an agents class

In [4]:
from typing import Any


class MyAgent:
    def __init__(self) -> None:
        #agent的提示词，用来描述agent的功能
        self.template =  """尽你最大可能回答下面问题，你将始终用中文回答. 你在必要时可以使用下面这些工具:
                    {tools}
                    Use the following format:
                    Question: the input question you must answer
                    Thought: you should always think about what to do
                    Action: the action to take, should be one of [{tool_names}]
                    Action Input: the input to the action
                    Observation: the result of the action
                    ... (this Thought/Action/Action Input/Observation can repeat N times)
                    Thought: I now know the final answer
                    Final Answer: the final answer to the original input question
                    Begin! 记住使用中文回答，如果你使用英文回答将回遭到惩罚.
                    Question: {input}
                    {agent_scratchpad}"""
        #定义一个openai的llm
        self.llm = ChatOpenAI(
            temperature=0,
            model="gpt-4-1106-preview"
        )
        #工具列表
        self.tools = self.MyAgentTool().tools()
        #agent的prompt
        self.prompt = self.MyTemplate(
            template=self.template,
            tools=self.tools,
            #输入变量和中间变量
            input_variables=["input", "intermediate_steps"],
        )

        #定义一个LLMChain
        self.llm_chain = LLMChain(
            llm=self.llm,
            prompt = self.prompt
        )
        #工具名称列表
        self.toolnames = [tool.name for tool in self.tools]
        #定义一个agent
        self.agent = LLMSingleActionAgent(
            llm_chain=self.llm_chain,
            allowed_tools=self.toolnames,
            output_parser=self.MyOutputParser(),
            
            stop=["\nObservation:"],
        )
    
    #运行agent
    def run(self, input: str) -> str:
        #创建一个agent执行器
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.agent, 
            tools=self.tools, 
            handle_parsing_errors=True,
            verbose=True
        )
        agent_executor.run(input=input)

    #自定义工具类
    class MyAgentTool:
        def __init__(self) -> None:
            self.serpapi = SerpAPIWrapper()
            
        def tools(self):
            return [
                Tool(
                    name="search",
                    description="适用于当你需要回答关于当前事件的问题时",
                    func=self.serpapi.run,
                )
            ]
    

    #自定义模版渲染类
    class MyTemplate(StringPromptTemplate):
        #渲染模版
        template: str
        #需要用到的工具
        tools:List[Tool]

        #格式化函数
        def format(self, **kwargs: Any) -> str:
            #获取中间步骤
            intermediate_steps = kwargs.pop("intermediate_steps")
            thoughts = ""
            for action, observation in intermediate_steps:
                thoughts += action.log
                thoughts += f"\nObservation: {observation}\nThought: "
            #将agent_scratchpad设置为该值
            kwargs["agent_scratchpad"] = thoughts
            # 从提供的工具列表中创建一个名为tools的变量
            kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
            #创建一个提供的工具名称列表
            kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
            return self.template.format(**kwargs)


    #自定义输出解析类
    class MyOutputParser(AgentOutputParser):
        #解析函数
        def parse(self, output: str) -> Union[AgentAction, AgentFinish]:
            #检查agent是否应该完成
            if "Final Answer:" in output:
                return AgentFinish(
                # 返回值通常始终是一个具有单个 `output` 键的字典。
                # It is not recommended to try anything else at the moment :)
                return_values={"output": output.split("Final Answer:")[-1].strip()},
                log=output,
                )
            #用正则解析出动作和动作输入
            regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
            match = re.search(regex, output, re.DOTALL)
            #如果没有匹配到则抛出异常
            if not match:
                raise OutputParserException(f"Could not parse LLM output: `{output}`")
            action = match.group(1).strip()
            action_input = match.group(2)
            # 返回操作和操作输入
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=output)

    

In [5]:
myagent = MyAgent()
myagent.run("比特币现在多少钱了？预计未来会涨吗？")

e:\教程\imooc\langchan_course\langchain_v0.1\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMSingleActionAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
e:\教程\imooc\langchan_course\langchain_v0.1\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: 为了回答比特币当前的价格，我需要查找最新的市场数据。至于预计未来是否会涨，这通常是基于市场分析和专家意见，我可以提供一些基本的市场趋势分析，但需要明确这不是投资建议。

Action: search
Action Input: 比特币当前价格


Observation:比特币当前价格为$63,970.50。 比特币的价格从00:00 (UTC+8) 上涨了+1.28%。 目前，比特币市值排名为第1 名，实时市值为$12,620.91亿，流通供应量为19,756,384 BTC，最大供应量为21,000,000 BTC。 我们会实时更新比特币/USD 的价格。
我现在知道了比特币当前的价格，但是我需要注意到这个价格是实时变动的，所以提供的价格可能在用户看到时已经有所变化。至于预计未来是否会涨，我需要查找一些最近的市场分析或者专家预测来提供一个大致的趋势。

Action: search
Action Input: 比特币价格预测 2023


Observation:2023年比特币价格预测表明，到年底，比特币的高点可能达到43,000美元，潜在低点为30,000美元。
我已经找到了比特币当前的价格，同样找到了一些专家对于2023年的价格预测。我现在可以给出最终回答。

Final Answer: 比特币当前的价格约为63,970.50美元，请注意这个价格是实时变动的。至于预计未来是否会涨，有预测指出，比特币在2023年的价格高点可能达到43,000美元，而潜在低点可能为30,000美元。这表明市场专家们预计比特币的价格可能会波动，并且提供的这些数据仅供参考，不构成具体的投资建议。

> Finished chain.
